In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [2]:
df=pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')

In [3]:
def encoder(df):
    from sklearn.preprocessing import LabelEncoder
    labelencoder = LabelEncoder()
    for col in df.columns:
        #if df[col].dtypes=='object':
        df[col]=labelencoder.fit_transform(df[col])
    return df

df=encoder(df)
test=encoder(test)

In [4]:
X = df.drop(columns = 'Exited')
Y = df['Exited']

In [5]:
Y.value_counts()

Exited
0    130113
1     34921
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.25, stratify = Y)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0,max_depth=1, random_state=0)
gb.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, n_estimators=1000,
                           random_state=0)

In [8]:
gb.score(X_test,y_test)

0.8618483240020359

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth = 18, n_estimators = 40, min_samples_split = 40)


In [10]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
eclf2 = VotingClassifier(estimators=[('rf', rf), ('gb', gb)],voting='soft')

In [11]:
eclf2.fit(X_train,y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=18,
                                                     min_samples_split=40,
                                                     n_estimators=40)),
                             ('gb',
                              GradientBoostingClassifier(learning_rate=1.0,
                                                         max_depth=1,
                                                         n_estimators=1000,
                                                         random_state=0))],
                 voting='soft')

In [12]:
eclf2.score(X_test,y_test)

0.8631571293535956

In [13]:
from sklearn import svm
clf=svm.SVC()
clf.fit(X_train,y_train)

SVC()

In [14]:
clf.score(X_test,y_test)

0.788409801497855

In [20]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C = 1)
#LogisticRegression(C = 1).fit(X_train,y_train)

In [21]:
eclf = VotingClassifier(estimators=[('rf', rf), ('gb', gb), ('lr',lr)],voting='soft')

In [22]:
eclf.fit(X_train,y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=18,
                                                     min_samples_split=40,
                                                     n_estimators=40)),
                             ('gb',
                              GradientBoostingClassifier(learning_rate=1.0,
                                                         max_depth=1,
                                                         n_estimators=1000,
                                                         random_state=0)),
                             ('lr', LogisticRegression(C=1))],
                 voting='soft')

In [23]:
eclf.score(X_test,y_test)

0.8585035992147168

In [24]:
predict = eclf.predict_proba(test)[:,1]

In [25]:
array = []
for i in test['id']:
    array.append(i + 165034)

In [26]:
submission = pd.DataFrame()
submission['id'] = array
submission['Exited'] = predict

In [27]:
submission.to_csv('submission.csv', index = False)

In [28]:
submission

,id,Exited
0,165034,0.086903
1,165035,0.683970
2,165036,0.135935
3,165037,0.322561
4,165038,0.481850
...,...,...
110018,275052,0.074157
110019,275053,0.162637
110020,275054,0.060571
110021,275055,0.109938
